<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/2%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C_%EC%A0%9C%EB%AA%85%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  # 1. KOSPI, KOSDAQ 의 full.code 를 매일 장마감후 수집! (3시반에 장마감)
#              3가지 부류 : (새로 상장, 폐지, 이상 상태)  표시되고, 실시간 업데이트 되도록, 
#                                                      단, 오늘까지 데이터는 pickle or json 에 저장해두고, 내일부터 하나씩 업데이트하는 방식!
#   종목명/ fullcode/ code/ type/ status

# 2. 1번의 코드 리스트를 기준으로 매일 주가 정보 업데이트
#    fullcode : 코드명
#    start.date : 수집 첫 시작일 : 1900.01.01 로 고정
#    end.date : 수집 마지막 일 : 어제 (오늘이 평일이라는 가정하에, 오늘 정보는, 오늘 밤 12시에 KRX 홈피에 업데이트 된다. - 교수님) 


# +)         모든 정보를 받은 후에, 업데이트를 해야한다. 
#            근데, 비효율적으로 이미 다운받은 데이터를 처음부터 받는게 아니라, 
#            어제 데이터? 하나만 업데이트 해나간다.

In [ ]:
import json
import pickle
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime

headers = {'User-Agent': 'Chrome/93.0.4577.82 Safari/537.36',}

data = {'mktsel': 'ALL',
        'typeNo': '0',
        'searchText': '',
        'bld': 'dbms/comm/finder/finder_stkisu',}

url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

r = requests.post(url, data, headers=headers)
j = json.loads(r.text)

df = pd.json_normalize(j['block1'])      # 이 부분은, html request를 보내서 데이터를 받아오는데, 보통 response 가 jason 형태입니다.
# 때문에 json 형태의 데이터를 json.loads(r.text) 로 파이썬에서 사용하는 딕셔너리 형태로 변환해준 다음에
# df = pd.json_normalize(j['block1'])
# 이 소스로, 해당 딕셔너리에서 block1가 키인 데이터를 가져와서 dataframe 형식으로 다시 변환해주는겁니다.

print(len(df))
print(df.iloc[1]['full_code'])

2584
KR7095570008


In [ ]:
df

,full_code,short_code,codeName,marketCode,marketName,marketEngName,ord1,ord2
0,KR7060310000,060310,3S,KSQ,코스닥,KOSDAQ,,16
1,KR7095570008,095570,AJ네트웍스,STK,유가증권,KOSPI,,16
2,KR7006840003,006840,AK홀딩스,STK,유가증권,KOSPI,,16
3,KR7054620000,054620,APS홀딩스,KSQ,코스닥,KOSDAQ,,16
4,KR7265520007,265520,AP시스템,KSQ,코스닥,KOSDAQ,,16
...,...,...,...,...,...,...,...,...
2579,KR7000542001,000547,흥국화재2우B,STK,유가증권,KOSPI,,16
2580,KR7000541003,000545,흥국화재우,STK,유가증권,KOSPI,,16
2581,KR7003280005,003280,흥아해운,STK,유가증권,KOSPI,,16
2582,KR7037440005,037440,희림,KSQ,코스닥,KOSDAQ,,16


In [ ]:
df = df.set_index('short_code')
full_code = df.loc['005930']['full_code']

strtDd = '19000101'
endDd = '20210113'

data2 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03501',
'searchType': '1',
'mktId': 'ALL',
'trdDd': '20210917',
'tboxisuCd_finder_stkisu0_1': '060310/3S',
'isuCd': 'KR7060310000',
'isuCd2': 'KR7005930003',
'codeNmisuCd_finder_stkisu0_1': '3S',
'param1isuCd_finder_stkisu0_1': 'ALL',
'strtDd': '20210910',
'endDd': '20210917',
'csvxls_isNo': 'false',}
        
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

r2 = requests.post(url2, data2, headers)
j2 = json.loads(r2.text)
df3 = pd.DataFrame(j2['output'])
df3

,ISU_SRT_CD,ISU_ABBRV,ISU_ABBRV_STR,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,060310,3S,"3S <em class =""up""></em>","3,705",1,85,2.35,16,231.56,792,4.68,0,0.00
1,095570,AJ네트웍스,"AJ네트웍스 <em class =""up""></em>","6,220",1,150,2.47,-,-,"6,089",1.02,210,3.38
2,006840,AK홀딩스,"AK홀딩스 <em class =""up""></em>","27,100",2,-150,-0.55,-,-,"50,471",0.54,400,1.48
3,054620,APS홀딩스,"APS홀딩스 <em class =""up""></em>","18,300",1,350,1.95,-,-,"8,135",2.25,0,0.00
4,265520,AP시스템,"AP시스템 <em class =""up""></em>","28,000",2,-50,-0.18,"1,736",16.13,"9,041",3.10,120,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,000547,흥국화재2우B,"흥국화재2우B <em class =""up""></em>","29,000",2,-350,-1.19,-,-,-,-,0,0.00
2400,000545,흥국화재우,"흥국화재우 <em class =""up""></em>","9,240",2,-90,-0.96,-,-,-,-,0,0.00
2401,003280,흥아해운,"흥아해운 <em class =""up""></em>","3,410",4,785,29.90,-,-,-,-,0,0.00
2402,037440,희림,"희림 <em class =""up""></em>","7,010",2,-40,-0.57,389,18.02,"4,034",1.74,150,2.14


In [ ]:
full_code = df.loc['005930']['full_code']

strtDd = '19000101'
endDd = '20210113'

data2 = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'searchType': '2',
    'mktId': 'ALL',
    'trdDd': '20210917',
    'tboxisuCd_finder_stkisu0_1': '005930/삼성전자',
    'isuCd': full_code,
    'isuCd2': full_code,
    'codeNmisuCd_finder_stkisu0_1': '삼성전자',
    'param1isuCd_finder_stkisu0_1': 'ALL',
    'strtDd': strtDd,
    'endDd': endDd,
    'csvxls_isNo': 'false',
}

url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

r2 = requests.post(url2, data2, headers)
j2 = json.loads(r2.text)
df2 = pd.DataFrame(j2['output'])
df2

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2021/01/13,"89,700",2,-900,-0.99,"3,166",28.33,"37,528",2.39,"1,416",1.58
1,2021/01/12,"90,600",2,-400,-0.44,"3,166",28.62,"37,528",2.41,"1,416",1.56
2,2021/01/11,"91,000",1,"2,200",2.48,"3,166",28.74,"37,528",2.42,"1,416",1.56
3,2021/01/08,"88,800",1,"5,900",7.12,"3,166",28.05,"37,528",2.37,"1,416",1.59
4,2021/01/07,"82,900",1,700,0.85,"3,166",26.18,"37,528",2.21,"1,416",1.71
...,...,...,...,...,...,...,...,...,...,...,...
6510,1995/05/08,"121,000",2,"-1,000",-0.82,-,-,-,-,-,-
6511,1995/05/06,"122,000",2,-500,-0.41,-,-,-,-,-,-
6512,1995/05/04,"122,500",2,"-1,000",-0.81,-,-,-,-,-,-
6513,1995/05/03,"123,500",1,"4,000",3.35,-,-,-,-,-,-


In [ ]:
# 1. csv 파일보다, pickle 이 파일 로딩 속도가 10배 정도 빠르다고 한다.

# 2. Pickle 파일은 pickle이라는 파이썬 고유 라이브러리에서 다뤄지는 데이터 포맷이다. 
#    매뉴얼에 따르면 Pickle은 계층적 구조를 가지는 python의 객체를 binary stream으로 전환(serialization)해준다고 한다. 
#    그렇기에 저장할 수 있는 데이터 구조에 제한이 없고 python-specific이기 때문에 처리속도가 빠르다는 것이다. 

# 3.   또한가지 유용한 경우는 list, dictionary등을 파일 그대로 저장하면 용량이 매우 커지는데 pickle을 사용하면 binary형태로 저장되기 때문에 용량이 매우 작아진다

####  하지만 이 포맷의 단점은 파이썬 내에서만 접근이 가능하다는 것과 csv처럼 데이터 전환 없이 미리보기를 할 수 없다는 것이다.

# 출처: https://gmnam.tistory.com/248 [Voyager]
# 출처: https://wikidocs.net/8929

In [ ]:
# 출처: https://m.blog.naver.com/dbstnrrud93/221872397008

# 피클은, ~~~~

In [ ]:
# 할 일
# 1. 데이터 저장 - (dataframe 을 pickle로 저장)
# 2. 자동화 - 일정 시간되면, 자동으로 실행되게끔



In [ ]:
datas = [df, df2, df3]                   # df, df2, df3 3개의 리스트 모두를 datas 라는 객체에 저장한다.

In [ ]:
datas

[               full_code codeName marketCode marketName marketEngName ord1 ord2
 short_code                                                                     
 060310      KR7060310000       3S        KSQ        코스닥        KOSDAQ        16
 095570      KR7095570008   AJ네트웍스        STK       유가증권         KOSPI        16
 006840      KR7006840003    AK홀딩스        STK       유가증권         KOSPI        16
 054620      KR7054620000   APS홀딩스        KSQ        코스닥        KOSDAQ        16
 265520      KR7265520007    AP시스템        KSQ        코스닥        KOSDAQ        16
 ...                  ...      ...        ...        ...           ...  ...  ...
 000547      KR7000542001  흥국화재2우B        STK       유가증권         KOSPI        16
 000545      KR7000541003    흥국화재우        STK       유가증권         KOSPI        16
 003280      KR7003280005     흥아해운        STK       유가증권         KOSPI        16
 037440      KR7037440005       희림        KSQ        코스닥        KOSDAQ        16
 238490      KR7238490007   

In [ ]:
with open('datas.pkl', 'wb') as f:        # 데이터 저장 : datas.pkl 이라는 파일에, 저장하고자 하는 데이터를 Binary 형태로 저장한다.   write binary
  pickle.dump(datas, f)
  f.close()

In [ ]:
with open('datas.pkl', 'rb') as f:        # 데이터 불러오기 : 변수 datas 에 저장했던 dataframe 이 new 라는 객체에 그대로 불러와진다.
  new = pickle.load(f)

In [ ]:
new

[               full_code codeName marketCode marketName marketEngName ord1 ord2
 short_code                                                                     
 060310      KR7060310000       3S        KSQ        코스닥        KOSDAQ        16
 095570      KR7095570008   AJ네트웍스        STK       유가증권         KOSPI        16
 006840      KR7006840003    AK홀딩스        STK       유가증권         KOSPI        16
 054620      KR7054620000   APS홀딩스        KSQ        코스닥        KOSDAQ        16
 265520      KR7265520007    AP시스템        KSQ        코스닥        KOSDAQ        16
 ...                  ...      ...        ...        ...           ...  ...  ...
 000547      KR7000542001  흥국화재2우B        STK       유가증권         KOSPI        16
 000545      KR7000541003    흥국화재우        STK       유가증권         KOSPI        16
 003280      KR7003280005     흥아해운        STK       유가증권         KOSPI        16
 037440      KR7037440005       희림        KSQ        코스닥        KOSDAQ        16
 238490      KR7238490007   

In [ ]:
# while 문으로 단순히 일정 시간마다 반복하도록 해서, 시간 체크해서 일정 시간마다 동작되도록 해야될 듯합니다

In [ ]:
# 자동화 방법 : 작업 스케줄러? 로 할 수 있다고 하는데, 어떻게 할지 모르겠습니다.
# 또는, 특정 조건을 입력해둬서, 파이썬이 계속 실행되는 상태에서, 조건에 맞을 때, 실행되도록! 하는 방법도!  But, 많이 비효율적인듯.

In [ ]:
# 전체 소스를 묶어주시거나, 이전에 작성한 것들을 함수로 묶어주시고 해당 함수 호출하는 방식으로 하셔도 됩니다
# 1900년 1월 1일부터 구하는건...음 크롤링쪽 자체를 수정해야할 듯 ??

In [ ]:

#from datetime import date
#import time

#today = '2021-09-24'
#while True:
#....if today == str(date.today()):
#........time.sleep(3600)
#........continue
#....today = str(date.today())

In [ ]:
# 문제는, colab으로는 어려울 것 같다.
# 여기서 부터는, idle 사용해서, 파일 형태로 따로 만들어야 할듯? 당연한 얘긴가?

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
res_path = '/content/drive/MyDrive/k-pro-folder'

# 이렇게 해서 경로를 지정해줬어요!
# 아 뭐가 다른게 아니라 그 drive mount는 내 구글 드라이브랑 연결? 시킨다는거고
# respath는 그냥 제가 경로 지정해준거예요!
# respath는 굳이 안하셔도 돼요!! 왜냐면 경로를 여러번 쓰기 귀찮아서 제가 변수화 한거구요
# drive mount 는 꼭 필요!!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
## Save pickle
with open(res_path + "df2.pickle","wb") as fw:
    pickle.dump(df2, fw)


In [ ]:
## Load pickle
# pickle로 저장되었던 데이터 출력 (어제까지의 데이터)
with open(res_path + "df2.pickle","rb") as fr:
    dataf = pickle.load(fr)
print(dataf)

# 그리고 이렇게 하면 볼 수 있어요!
# 오!!!! 대박!!! 생겼다!!!

          TRD_DD TDD_CLSPRC FLUC_TP_CD  ...   PBR    DPS DVD_YLD
0     2021/01/13     89,700          2  ...  2.39  1,416    1.58
1     2021/01/12     90,600          2  ...  2.41  1,416    1.56
2     2021/01/11     91,000          1  ...  2.42  1,416    1.56
3     2021/01/08     88,800          1  ...  2.37  1,416    1.59
4     2021/01/07     82,900          1  ...  2.21  1,416    1.71
...          ...        ...        ...  ...   ...    ...     ...
6510  1995/05/08    121,000          2  ...     -      -       -
6511  1995/05/06    122,000          2  ...     -      -       -
6512  1995/05/04    122,500          2  ...     -      -       -
6513  1995/05/03    123,500          1  ...     -      -       -
6514  1995/05/02    119,500          2  ...     -      -       -

[6515 rows x 11 columns]


In [ ]:
df

,full_code,codeName,marketCode,marketName,marketEngName,ord1,ord2
short_code,,,,,,,
060310,KR7060310000,3S,KSQ,코스닥,KOSDAQ,,16
095570,KR7095570008,AJ네트웍스,STK,유가증권,KOSPI,,16
006840,KR7006840003,AK홀딩스,STK,유가증권,KOSPI,,16
054620,KR7054620000,APS홀딩스,KSQ,코스닥,KOSDAQ,,16
265520,KR7265520007,AP시스템,KSQ,코스닥,KOSDAQ,,16
...,...,...,...,...,...,...,...
000547,KR7000542001,흥국화재2우B,STK,유가증권,KOSPI,,16
000545,KR7000541003,흥국화재우,STK,유가증권,KOSPI,,16
003280,KR7003280005,흥아해운,STK,유가증권,KOSPI,,16


In [ ]:
df2

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2021/01/13,"89,700",2,-900,-0.99,"3,166",28.33,"37,528",2.39,"1,416",1.58
1,2021/01/12,"90,600",2,-400,-0.44,"3,166",28.62,"37,528",2.41,"1,416",1.56
2,2021/01/11,"91,000",1,"2,200",2.48,"3,166",28.74,"37,528",2.42,"1,416",1.56
3,2021/01/08,"88,800",1,"5,900",7.12,"3,166",28.05,"37,528",2.37,"1,416",1.59
4,2021/01/07,"82,900",1,700,0.85,"3,166",26.18,"37,528",2.21,"1,416",1.71
...,...,...,...,...,...,...,...,...,...,...,...
6510,1995/05/08,"121,000",2,"-1,000",-0.82,-,-,-,-,-,-
6511,1995/05/06,"122,000",2,-500,-0.41,-,-,-,-,-,-
6512,1995/05/04,"122,500",2,"-1,000",-0.81,-,-,-,-,-,-
6513,1995/05/03,"123,500",1,"4,000",3.35,-,-,-,-,-,-


In [ ]:
df3

,ISU_SRT_CD,ISU_ABBRV,ISU_ABBRV_STR,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,060310,3S,"3S <em class =""up""></em>","3,705",1,85,2.35,16,231.56,792,4.68,0,0.00
1,095570,AJ네트웍스,"AJ네트웍스 <em class =""up""></em>","6,220",1,150,2.47,-,-,"6,089",1.02,210,3.38
2,006840,AK홀딩스,"AK홀딩스 <em class =""up""></em>","27,100",2,-150,-0.55,-,-,"50,471",0.54,400,1.48
3,054620,APS홀딩스,"APS홀딩스 <em class =""up""></em>","18,300",1,350,1.95,-,-,"8,135",2.25,0,0.00
4,265520,AP시스템,"AP시스템 <em class =""up""></em>","28,000",2,-50,-0.18,"1,736",16.13,"9,041",3.10,120,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,000547,흥국화재2우B,"흥국화재2우B <em class =""up""></em>","29,000",2,-350,-1.19,-,-,-,-,0,0.00
2400,000545,흥국화재우,"흥국화재우 <em class =""up""></em>","9,240",2,-90,-0.96,-,-,-,-,0,0.00
2401,003280,흥아해운,"흥아해운 <em class =""up""></em>","3,410",4,785,29.90,-,-,-,-,0,0.00
2402,037440,희림,"희림 <em class =""up""></em>","7,010",2,-40,-0.57,389,18.02,"4,034",1.74,150,2.14


In [ ]:
df3.columns

Index(['ISU_SRT_CD', 'ISU_ABBRV', 'ISU_ABBRV_STR', 'TDD_CLSPRC', 'FLUC_TP_CD',
       'CMPPREVDD_PRC', 'FLUC_RT', 'EPS', 'PER', 'BPS', 'PBR', 'DPS',
       'DVD_YLD'],
      dtype='object')

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ISU_SRT_CD     2404 non-null   object
 1   ISU_ABBRV      2404 non-null   object
 2   ISU_ABBRV_STR  2404 non-null   object
 3   TDD_CLSPRC     2404 non-null   object
 4   FLUC_TP_CD     2404 non-null   object
 5   CMPPREVDD_PRC  2404 non-null   object
 6   FLUC_RT        2404 non-null   object
 7   EPS            2404 non-null   object
 8   PER            2404 non-null   object
 9   BPS            2404 non-null   object
 10  PBR            2404 non-null   object
 11  DPS            2404 non-null   object
 12  DVD_YLD        2404 non-null   object
dtypes: object(13)
memory usage: 244.3+ KB


In [ ]:
profile_file = open("profile.pickle", "wb")

In [ ]:
profile = [df, df2, df3]

In [ ]:
print(profile)

[               full_code codeName marketCode marketName marketEngName ord1 ord2
short_code                                                                     
060310      KR7060310000       3S        KSQ        코스닥        KOSDAQ        16
095570      KR7095570008   AJ네트웍스        STK       유가증권         KOSPI        16
006840      KR7006840003    AK홀딩스        STK       유가증권         KOSPI        16
054620      KR7054620000   APS홀딩스        KSQ        코스닥        KOSDAQ        16
265520      KR7265520007    AP시스템        KSQ        코스닥        KOSDAQ        16
...                  ...      ...        ...        ...           ...  ...  ...
000547      KR7000542001  흥국화재2우B        STK       유가증권         KOSPI        16
000545      KR7000541003    흥국화재우        STK       유가증권         KOSPI        16
003280      KR7003280005     흥아해운        STK       유가증권         KOSPI        16
037440      KR7037440005       희림        KSQ        코스닥        KOSDAQ        16
238490      KR7238490007       힘스      

In [ ]:
pickle.dump(profile, profile_file)

In [ ]:
profile_file.close()

In [1]:
!git clone https://github.com/baeksumin/k-project.git

Cloning into 'k-project'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
